In [109]:
from gensim.models import Word2Vec
from ast import literal_eval
import pandas as pd
import itertools
import pickle
import MeCab
import re

# Korean 10-k

In [69]:
morphs_to_keep =['NNG', 'NNP', 'NNBC', 'NR', 'VV', 'VA', 'SL']

In [39]:
df_2015 = pd.read_csv('C:\\Users\\sylim2357\\Documents\\git\\korean_financial_news_scrapers\\jupyter_notebook\\kospi200_2015_2019_parsed.csv', encoding='utf-8')
df_2015['Relevant_Sentences'] = [literal_eval(s) for s in df_2015['Relevant_Sentences']]
df_2011 = pd.read_csv('C:\\Users\\sylim2357\\Documents\\git\\korean_financial_news_scrapers\\jupyter_notebook\\kospi200_2011_2015_parsed.csv', encoding='utf-8')
df_2011['Relevant_Sentences'] = [literal_eval(s) for s in df_2011['Relevant_Sentences']]
df = df_2011.append(df_2015)
df = df.reset_index()

In [73]:
def mecab_tokenize(sentence, morphs_to_keep):
    t = MeCab.Tagger()
    return [tuple(re.split(',', re.sub('\t', ',', s)))[0] for s in t.parse(sentence).split('\n') if (s!='') & ('EOS' not in s) & (any(substring in s for substring in morphs_to_keep)
)]

df['w2v_corpus'] = None
for idx, row in df.iterrows():
    df.at[idx, 'w2v_corpus'] = [mecab_tokenize(sentence, morphs_to_keep) for sentence in row['Relevant_Sentences']]

In [96]:
corpus = list(itertools.chain.from_iterable(df['w2v_corpus']))

In [100]:
w2v_model = Word2Vec(corpus, size=100, window=5, min_count=1, workers=4)

In [101]:
w2v_model.most_similar('예상')

C:\Users\sylim2357\anaconda3\envs\future-tense-mining\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('전망', 0.7743479013442993),
 ('기대', 0.7218489646911621),
 ('보입니다', 0.6938493251800537),
 ('상되', 0.647679328918457),
 ('보여집니다', 0.6302053332328796),
 ('흥국', 0.5328930020332336),
 ('점진', 0.5217318534851074),
 ('보이', 0.5190744400024414),
 ('급속도', 0.5074462890625),
 ('보여', 0.506676435470581)]

In [102]:
w2v_model.most_similar('할')

C:\Users\sylim2357\anaconda3\envs\future-tense-mining\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('하', 0.5327817797660828),
 ('맡', 0.5041417479515076),
 ('가표', 0.4622926414012909),
 ('기할', 0.4615386128425598),
 ('보로서', 0.438471257686615),
 ('해내', 0.421520471572876),
 ('OBD', 0.401702880859375),
 ('다질', 0.40035802125930786),
 ('도달', 0.39226865768432617),
 ('융통', 0.37887415289878845)]

In [103]:
w2v_model.most_similar('예정')

C:\Users\sylim2357\anaconda3\envs\future-tense-mining\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('계획', 0.6003089547157288),
 ('중임', 0.4524746537208557),
 ('서관', 0.44184744358062744),
 ('추가', 0.4343995451927185),
 ('완료', 0.4327957332134247),
 ('후', 0.4279869496822357),
 ('본격', 0.42749524116516113),
 ('EPCM', 0.4203566312789917),
 ('조기', 0.41347968578338623),
 ('value', 0.41292276978492737)]

In [107]:
w2v_model.most_similar('향후')

C:\Users\sylim2357\anaconda3\envs\future-tense-mining\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('점진', 0.5667904019355774),
 ('최근', 0.5637996196746826),
 ('성장', 0.504819393157959),
 ('지속', 0.47213542461395264),
 ('민란', 0.46434926986694336),
 ('봐도', 0.46375423669815063),
 ('시장', 0.4494169056415558),
 ('고성장', 0.4410852789878845),
 ('로', 0.43820831179618835),
 ('전망', 0.4380779564380646)]

In [108]:
w2v_model.most_similar('기대')

C:\Users\sylim2357\anaconda3\envs\future-tense-mining\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('예상', 0.7218489050865173),
 ('전망', 0.6744959354400635),
 ('보입니다', 0.5847667455673218),
 ('보여집니다', 0.5782133340835571),
 ('견인', 0.51365727186203),
 ('점진', 0.5090121030807495),
 ('외형', 0.5074797868728638),
 ('대폭', 0.5000908970832825),
 ('보여', 0.49400579929351807),
 ('그쳤', 0.48503172397613525)]

# AIHUB

In [124]:
with open('D:\\data\\text\\AIHub\\aihub_tokenized.pkl', 'rb') as f:
    a = pickle.load(f)

In [117]:
aihub_corpus = []
for sentence in a:
    aihub_corpus += [[word[0] for word in sentence if word[1] in morphs_to_keep]]

In [120]:
w2v_model_aihub = Word2Vec(aihub_corpus, size=100, window=5, min_count=1, workers=4)

In [121]:
w2v_model_aihub.most_similar('예상')

C:\Users\sylim2357\anaconda3\envs\future-tense-mining\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('전망', 0.8558336496353149),
 ('예측', 0.6863952875137329),
 ('내다보', 0.6690541505813599),
 ('관측', 0.6194316148757935),
 ('예년', 0.5847278833389282),
 ('최소', 0.5529088973999023),
 ('당장', 0.5491597652435303),
 ('기대', 0.5437719821929932),
 ('빠르', 0.5378214120864868),
 ('향후', 0.535881519317627)]

In [122]:
w2v_model_aihub.most_similar('예정')

C:\Users\sylim2357\anaconda3\envs\future-tense-mining\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('순차', 0.6893970966339111),
 ('계획', 0.6597926616668701),
 ('완료', 0.6458877921104431),
 ('방침', 0.564749002456665),
 ('정식', 0.5447733998298645),
 ('마무리', 0.5409709215164185),
 ('진행', 0.5268769860267639),
 ('날짜', 0.5177264213562012),
 ('추가', 0.5158536434173584),
 ('본격', 0.5139623284339905)]

In [123]:
w2v_model_aihub.most_similar('향후')

C:\Users\sylim2357\anaconda3\envs\future-tense-mining\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('장기', 0.647159218788147),
 ('지속', 0.5882102251052856),
 ('전망', 0.5777440667152405),
 ('예상', 0.535881519317627),
 ('확대', 0.5164848566055298),
 ('실질', 0.5093995332717896),
 ('현재', 0.5044463872909546),
 ('방침', 0.5027302503585815),
 ('최근', 0.4990479648113251),
 ('구체', 0.49387091398239136)]